In [ ]:
import tempfile
import kale.workflow_objects
import kale.workflow_widgets

In [ ]:
stress_batch_template = """
#!/bin/bash
#PBS -l nodes=1
#PBS -l nice=10
#PBS -j oe
#PBS -q default
#PBS -N StressTest{num_procs}_{duration}
#PBS -o /opt/kale/kale/examples/batch_logs/stress{num_procs}_{duration}.log
#PBS -r n
cd $PBS_O_WORKDIR
stress -c {num_procs} -t {duration}
"""

In [ ]:
rm -f ../batch_scripts/tmp*

In [ ]:
def batch_task_from_string(name, batch_string, poll_interval=45):
    with tempfile.NamedTemporaryFile('w',dir='../batch_scripts', delete=False) as tmp:
        tmp.write(batch_string)
    
    return kale.workflow_objects.BatchTask(
        name=name,
        batch_script=tmp.name,
        poll_interval=poll_interval,
    )

In [ ]:
def create_stress_task(num_procs, duration=10):
    return batch_task_from_string(
        'stress{}_{}'.format(num_procs, duration),
        stress_batch_template.format(
            num_procs=num_procs,
            duration=duration
        )
    )

In [ ]:
t0 = create_stress_task(1, 30)
t1 = create_stress_task(2, 45)

In [ ]:
wf = kale.workflow_objects.Workflow('Stress WF')
wf.add_task(t0)
wf.add_task(t1, dependencies=[t0])

In [ ]:
wpw = kale.workflow_widgets.WorkerPoolWidget()
wpw

In [ ]:
kale.workflow_widgets.WorkflowWidget(wf, wpw)